In [164]:
import pandas as pd
import numpy as np

In [165]:
url = 'https://raw.githubusercontent.com/lenhattung/SP26-AI1904-DAP391m/refs/heads/main/lab_part_2/01%20-%20Linear%20and%20logistic%20regression/titanic.csv'
df = pd.read_csv(url)

In [166]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  1309 non-null   int64  
 1   pclass        1309 non-null   int64  
 2   survived      1309 non-null   int64  
 3   name          1309 non-null   object 
 4   sex           1309 non-null   int64  
 5   age           1046 non-null   float64
 6   sibsp         1309 non-null   int64  
 7   parch         1309 non-null   int64  
 8   ticket        1309 non-null   object 
 9   fare          1308 non-null   float64
 10  cabin         295 non-null    object 
 11  embarked      1307 non-null   object 
 12  boat          486 non-null    object 
 13  body          121 non-null    float64
 14  home.dest     745 non-null    object 
dtypes: float64(3), int64(6), object(6)
memory usage: 153.5+ KB


,passenger_id,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,1,"Allen, Miss. Elisabeth Walton",1,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,2,1,1,"Allison, Master. Hudson Trevor",0,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1,0,"Allison, Miss. Helen Loraine",1,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1,0,"Allison, Mr. Hudson Joshua Creighton",0,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",1,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [167]:
df['age'] = df['age'].fillna(df['age'].median()).round(0).astype('int64')
df['fare'] = df['fare'].fillna(df['fare'].mean()).round(2).astype('float64')
X = df.drop(['cabin', 'embarked','boat', 'body', 'home.dest', 'name', 'ticket', 'passenger_id','survived'], axis = 1)
y = df['survived']
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  1309 non-null   int64  
 1   pclass        1309 non-null   int64  
 2   survived      1309 non-null   int64  
 3   name          1309 non-null   object 
 4   sex           1309 non-null   int64  
 5   age           1309 non-null   int64  
 6   sibsp         1309 non-null   int64  
 7   parch         1309 non-null   int64  
 8   ticket        1309 non-null   object 
 9   fare          1309 non-null   float64
 10  cabin         295 non-null    object 
 11  embarked      1307 non-null   object 
 12  boat          486 non-null    object 
 13  body          121 non-null    float64
 14  home.dest     745 non-null    object 
dtypes: float64(2), int64(7), object(6)
memory usage: 153.5+ KB


In [168]:
X = sm.add_constant(X)
model_sm = sm.OLS(y, X)
model_sm.fit().summary()
X

,const,pclass,sex,age,sibsp,parch,fare
0,1.0,1,1,29,0,0,211.34
1,1.0,1,0,1,1,2,151.55
2,1.0,1,1,2,1,2,151.55
3,1.0,1,0,30,1,2,151.55
4,1.0,1,1,25,1,2,151.55
...,...,...,...,...,...,...,...
1304,1.0,3,1,14,1,0,14.45
1305,1.0,3,1,28,1,0,14.45
1306,1.0,3,0,26,0,0,7.22
1307,1.0,3,0,27,0,0,7.22


In [169]:
# Không dùng thư viện cross val score mà code chay để hiểu bản chất
# Dung strafield
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
X.drop('const', axis = 1, inplace = True)
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
fold_score = []
print(f"{'Fold':<5} | {'Train Size':<10} | {'Val Size':<10} | {'Acc':<10}")
print("-" * 45)

for i, (train_index, val_index) in enumerate(kf.split(X, y)) :
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    fold_score.append(score)
    print(f"{i+1:<5} | {len(train_index):<10} | {len(val_index):<10} | {score:.4f}")

print("-" * 45)
print(f"Average Accuracy: {np.mean(fold_score):.4f}")
for i, j in zip(y_val, y_pred) :
    print(f'actual : {i} predict : {j}')

Fold  | Train Size | Val Size   | Acc       
---------------------------------------------
1     | 1047       | 262        | 0.7710
2     | 1047       | 262        | 0.8053
3     | 1047       | 262        | 0.7748
4     | 1047       | 262        | 0.8206
5     | 1048       | 261        | 0.7241
---------------------------------------------
Average Accuracy: 0.7792
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 0
actual : 0 predict : 1
actual : 1 predict : 0
actual : 1 predict : 1
actual : 1 predict : 0
actual : 0 predict : 0
actual : 1 predict : 0
actual : 0 predict : 0
actual : 0 predict : 1
actual : 1 predict : 0
actual : 1 predict : 0
actual : 1 predict : 0
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 0
actual : 1 predict : 1
actual : 0 predict : 0
actual : 1 predict : 0
actual : 1 p

In [170]:
# Sử dụng thư viện cross val score cho nhanh
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
kf = RepeatedKFold(n_splits = 5, n_repeats=10, random_state = 42)
model = RandomForestClassifier()
score = cross_val_score(model, X, y, cv = kf, scoring = 'accuracy')
print(score)
print(score.mean())
# rkf do chinh xac cao hon

[0.77099237 0.79389313 0.79389313 0.79770992 0.75862069 0.75954198
 0.7480916  0.80534351 0.82061069 0.78927203 0.80534351 0.80152672
 0.79007634 0.76717557 0.7816092  0.83587786 0.76335878 0.79007634
 0.7480916  0.81226054 0.7519084  0.75954198 0.79389313 0.78625954
 0.81226054 0.77099237 0.79007634 0.80152672 0.77480916 0.80076628
 0.80152672 0.80152672 0.77099237 0.80916031 0.7394636  0.76717557
 0.77480916 0.77480916 0.83206107 0.76245211 0.76335878 0.76335878
 0.79007634 0.78625954 0.83141762 0.75572519 0.80534351 0.77480916
 0.76335878 0.79310345]
0.7847237577140185


In [171]:
# Sử dụng thư viện cross val score cho nhanh
# acuracy tang khi du lieu duoc can bang hon (hen xui, co the giam neu du lieu thuc su khong co y nghia)
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
model = RandomForestClassifier()
score = cross_val_score(model, X, y, cv = kf, scoring = 'accuracy')
print(score)
print(score.mean())


[0.75954198 0.82442748 0.79389313 0.81679389 0.78927203]
0.7967857038401919


In [172]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
kf = RepeatedStratifiedKFold(n_splits = 5,n_repeats=10, random_state = 42)
fold_score = []
print(f"{'Fold':<5} | {'Train Size':<10} | {'Val Size':<10} | {'Acc':<10}")
print("-" * 45)

for i, (train_index, val_index) in enumerate(kf.split(X, y)) :
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    fold_score.append(score)
    print(f"{i+1:<5} | {len(train_index):<10} | {len(val_index):<10} | {score:.4f}")

print("-" * 45)
print(f"Average Accuracy: {np.mean(fold_score):.4f}")
for i, j in zip(y_val, y_pred) :
    print(f'actual : {i} predict : {j}')
print(score.mean())

Fold  | Train Size | Val Size   | Acc       
---------------------------------------------
1     | 1047       | 262        | 0.7710
2     | 1047       | 262        | 0.7977
3     | 1047       | 262        | 0.7710
4     | 1047       | 262        | 0.7863
5     | 1048       | 261        | 0.8008
6     | 1047       | 262        | 0.7786
7     | 1047       | 262        | 0.7977
8     | 1047       | 262        | 0.7634
9     | 1047       | 262        | 0.7710
10    | 1048       | 261        | 0.8008
11    | 1047       | 262        | 0.7939
12    | 1047       | 262        | 0.7824
13    | 1047       | 262        | 0.7939
14    | 1047       | 262        | 0.7748
15    | 1048       | 261        | 0.7931
16    | 1047       | 262        | 0.7977
17    | 1047       | 262        | 0.7786
18    | 1047       | 262        | 0.7595
19    | 1047       | 262        | 0.7863
20    | 1048       | 261        | 0.8008
21    | 1047       | 262        | 0.7672
22    | 1047       | 262        | 0.7748
23    |

AttributeError: 'float' object has no attribute 'mean'

In [174]:


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
kf = RepeatedStratifiedKFold(n_splits = 5,n_repeats=10, random_state = 42)
fold_score = []
print(f"{'Fold':<5} | {'Train Size':<10} | {'Val Size':<10} | {'Acc':<10}")
print("-" * 45)

for i, (train_index, val_index) in enumerate(kf.split(X, y)) :
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    fold_score.append(score)
    print(f"{i+1:<5} | {len(train_index):<10} | {len(val_index):<10} | {score:.4f}")

print("-" * 45)
print(f"Average Accuracy: {np.mean(fold_score):.4f}")
for i, j in zip(y_val, y_pred) :
    print(f'actual : {i} predict : {j}')
print(np.mean(fold_score))

Fold  | Train Size | Val Size   | Acc       
---------------------------------------------
1     | 1047       | 262        | 0.7710
2     | 1047       | 262        | 0.7977
3     | 1047       | 262        | 0.7710
4     | 1047       | 262        | 0.7863
5     | 1048       | 261        | 0.8008
6     | 1047       | 262        | 0.7786
7     | 1047       | 262        | 0.7977
8     | 1047       | 262        | 0.7634
9     | 1047       | 262        | 0.7710
10    | 1048       | 261        | 0.8008
11    | 1047       | 262        | 0.7939
12    | 1047       | 262        | 0.7824
13    | 1047       | 262        | 0.7939
14    | 1047       | 262        | 0.7748
15    | 1048       | 261        | 0.7931
16    | 1047       | 262        | 0.7977
17    | 1047       | 262        | 0.7786
18    | 1047       | 262        | 0.7595
19    | 1047       | 262        | 0.7863
20    | 1048       | 261        | 0.8008
21    | 1047       | 262        | 0.7672
22    | 1047       | 262        | 0.7748
23    |

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Train thử 1 cây quyết định
dt = DecisionTreeClassifier()
dt.fit(X, y)

# Xem độ quan trọng của từng cột
import pandas as pd
importance = pd.DataFrame({
    'Feature': X.columns,
    'Score': dt.feature_importances_
}).sort_values(by='Score', ascending=False)

print("--- KIỂM TRA ĐỘ QUAN TRỌNG ---")
print(importance.head(5))

--- KIỂM TRA ĐỘ QUAN TRỌNG ---
  Feature     Score
1     sex  0.308789
5    fare  0.292220
2     age  0.215951
0  pclass  0.104397
3   sibsp  0.053825
